# Preparing Hurricane Sandy data for REOSSP-RHP (Notebook)

**What this notebook does (step-by-step)**

1. Shows where to download Hurricane Sandy track data (NOAA / NHC) and how to save it as `sandy_track.csv`.
2. Loads the track CSV and extracts **P = 29** target points along the track.
3. Defines the two ground stations used in the paper (Svalbard & Boecillo).
4. Generates simple circular Walker‑delta satellite positions (toy model) for K satellites.
5. Computes simple visibility matrices `V`, `W`, and `H` (approximations) and saves a data file
   `reossp_sandy_data.pkl` that can be loaded by the toy solver scripts (Pyomo / Gurobi).

**Notes & limitations**

- This notebook provides a _practical, runnable_ pipeline to prepare inputs. It is **not** a replacement
  for full SGP4 propagation or precise delta‑v calculations used in the paper. For accurate production
  experiments you should replace the simple orbital model with SGP4/poliastro and compute transfer costs
  with proper orbital mechanics formulas.

---

To run: upload the file `sandy_track.csv` (if not already present) into the same directory as this notebook.
See the first code cell for the expected CSV format (columns: `time,lat,lon` or similar).


## Step 0 — Download Hurricane Sandy track data

Download the 'best track' data for Hurricane Sandy (2012) from the NHC/NOAA or use the Tropical Cyclone Report:

- NHC Tropical Cyclone Report (Sandy): https://www.nhc.noaa.gov/data/tcr/AL182012_Sandy.pdf

If you prefer a ready CSV, there are several community datasets of hurricane tracks (or convert the TCR table to CSV).

**Expected CSV format (example header):**

`time,latitude,longitude`  — where `time` can be any parsable timestamp and `latitude`/`longitude` are decimal degrees.

Save the file as `sandy_track.csv` in the same directory as this notebook.


In [ ]:
# Step 1 — Load Sandy track CSV and preview
import os, sys
import pandas as pd

csv_path = "sandy_track.csv"
if not os.path.exists(csv_path):
    print("File 'sandy_track.csv' not found in the current folder.")
    print("Please download the Sandy track (NHC/NOAA) and save it as 'sandy_track.csv' with columns like: time,latitude,longitude")
else:
    df = pd.read_csv(csv_path)
    # try to standardize column names
    colmap = {c.lower(): c for c in df.columns}
    # common variants: 'lat','latitude'; 'lon','longitude'; 'time','datetime'
    lat_col = colmap.get('latitude', colmap.get('lat'))
    lon_col = colmap.get('longitude', colmap.get('lon'))
    time_col = colmap.get('time', colmap.get('datetime', colmap.get('date')))

    if lat_col is None or lon_col is None:
        print("CSV does not contain latitude/longitude columns. Found columns:", df.columns.tolist())
    else:
        df = df.rename(columns={lat_col: 'lat', lon_col: 'lon'})
        if time_col is not None:
            df = df.rename(columns={time_col: 'time'})
            df['time'] = pd.to_datetime(df['time'], errors='coerce')
        print('Loaded sandy_track.csv — preview:')
        display(df.head())
        print('\\nTotal rows:', len(df))


In [ ]:
# Step 2 — Create P = 29 target points from the track
import math

if not os.path.exists(csv_path):
    print("Please provide sandy_track.csv first.")
else:
    N_targets = 29
    # If the CSV has roughly 29 entries (6h spacing), take them directly; otherwise sample evenly along the track.
    rows = len(df)
    if rows >= N_targets:
        indices = [round(i * (rows-1) / (N_targets-1)) for i in range(N_targets)]
    else:
        # if fewer rows than 29, repeat last or interpolate — here we just reuse endpoints
        indices = list(range(rows)) + [rows-1]*(N_targets-rows)
        indices = indices[:N_targets]

    targets = []
    for idx in indices:
        r = df.iloc[idx]
        targets.append({'p': len(targets)+1, 'lat': float(r['lat']), 'lon': float(r['lon']), 'time': r.get('time', None)})
    print(f"Created {len(targets)} targets (P={N_targets}). First targets:")
    for t in targets[:5]:
        print(t)


In [ ]:
# Step 3 — Define ground stations used in the paper
ground_stations = {
    1: {'name': 'Svalbard', 'lat': 78.23, 'lon': 15.41},
    2: {'name': 'Boecillo', 'lat': 41.54, 'lon': -4.70}
}
print('Ground stations:')
for k,v in ground_stations.items():
    print(k, v)


In [ ]:
# Step 4 — Simple circular Walker-delta orbit generator (toy)
# This is _not_ SGP4 — it's a simple Keplerian circular orbit generator for demonstration.
import numpy as np

def latlon_to_ecef(lat_deg, lon_deg, alt_km=0.0, R_earth_km=6371.0):
    # Convert geodetic lat/lon to ECEF (approximate, assuming spherical Earth)
    lat = np.deg2rad(lat_deg)
    lon = np.deg2rad(lon_deg)
    r = R_earth_km + alt_km
    x = r * np.cos(lat) * np.cos(lon)
    y = r * np.cos(lat) * np.sin(lon)
    z = r * np.sin(lat)
    return np.array([x,y,z])

def circular_orbit_positions(incl_deg, r_km, n_points, raan_deg=0.0, arglat0_deg=0.0):
    # produce ECI-like vectors of a circular orbit (toy)
    incl = np.deg2rad(incl_deg)
    raan = np.deg2rad(raan_deg)
    arglat0 = np.deg2rad(arglat0_deg)
    positions = []
    for m in range(n_points):
        theta = arglat0 + 2*np.pi*m/n_points
        x_orb = r_km * np.cos(theta)
        y_orb = r_km * np.sin(theta)
        z_orb = 0.0
        R_x = np.array([[1,0,0],[0,np.cos(incl),-np.sin(incl)],[0,np.sin(incl),np.cos(incl)]])
        R_z = np.array([[np.cos(raan), -np.sin(raan),0],[np.sin(raan),np.cos(raan),0],[0,0,1]])
        pos = R_z.dot(R_x.dot(np.array([x_orb,y_orb,z_orb])))
        positions.append(pos)
    return np.array(positions)

# Example: create K=4 Walker-delta like satellites (toy)
K = 4
alt_km = 709.0  # as in the paper (approximate Earth radius+alt)
r_km = 6371.0 + alt_km
inclination = 98.18
T = max(1, len(targets))
sat_positions = {}
for k in range(1, K+1):
    arglat0 = (k-1) * (360.0 / K)
    pos = circular_orbit_positions(inclination, r_km, T, raan_deg=0.0, arglat0_deg=arglat0)
    sat_positions[k] = pos

print('Generated simple circular orbit positions for K=', K, 'satellites, T=', T)


In [ ]:
# Step 5 — Compute simple visibility matrices (toy approximations)
import numpy as np
from math import acos

FOV_deg = 45.0
COM_range_deg = 120.0
R_earth = 6371.0

def angle_between(u,v):
    dot = np.dot(u,v)
    nu = np.linalg.norm(u); nv = np.linalg.norm(v)
    if nu*nv == 0: return np.pi
    cosang = np.clip(dot/(nu*nv), -1.0, 1.0)
    return np.arccos(cosang)

V = {}
W = {}
H = {}

S = 8
targets_per_stage = math.ceil(len(targets)/S)
stage_indices = []
for s_idx in range(1, S+1):
    start = (s_idx-1)*targets_per_stage
    end = min(len(targets), s_idx*targets_per_stage)
    stage_indices.append(list(range(start, end)))

J = {}
for s_idx in range(1, S+1):
    for k in range(1, K+1):
        J[(s_idx,k)] = [1]

for s_idx in range(1, S+1):
    idx_list = stage_indices[s_idx-1]
    if not idx_list:
        continue
    for k in range(1, K+1):
        for local_t_idx, global_target_idx in enumerate(idx_list):
            t = local_t_idx + 1
            sat_pos = sat_positions[k][global_target_idx % T]
            tgt = targets[global_target_idx]
            tgt_pos = latlon_to_ecef(tgt['lat'], tgt['lon'], alt_km=0.0)
            for p in [tgt['p']]:
                ang = angle_between(sat_pos, tgt_pos)
                ang_deg = np.rad2deg(ang)
                if ang_deg <= FOV_deg:
                    V[(s_idx,k,t,1,p)] = 1
            for g_idx, ginfo in ground_stations.items():
                gs_pos = latlon_to_ecef(ginfo['lat'], ginfo['lon'], alt_km=0.0)
                ang_g = angle_between(sat_pos, gs_pos)
                ang_g_deg = np.rad2deg(ang_g)
                if ang_g_deg <= COM_range_deg:
                    W[(s_idx,k,t,1,g_idx)] = 1
            sun_dir = np.array([1.0,0.0,0.0])
            if np.rad2deg(angle_between(sat_pos, sun_dir)) < 90.0:
                H[(s_idx,k,t,1)] = 1
            else:
                H[(s_idx,k,t,1)] = 0

print("Computed V, W, H sizes:", len(V), len(W), len(H))


In [ ]:
# Step 6 — Save data dictionary to a file (pickle) that the solver script can load
import pickle
data = {
    "S": S,
    "K": list(range(1, K+1)),
    "P": list(range(1, len(targets)+1)),
    "G": list(ground_stations.keys()),
    "T_s": {s_idx: list(range(1, len(stage_indices[s_idx-1])+1)) for s_idx in range(1, S+1)},
    "J": J,
    "V": V,
    "W": W,
    "H": H,
    "Dobs": 5.0,
    "Dcomm": 4.0,
    "Bobs": 1.0,
    "Bcomm": 0.8,
    "Bcharge": 2.0,
    "Btime": 0.2,
    "Dmin": 0.0,
    "Dmax": 500.0,
    "Bmin": 0.0,
    "Bmax": 1000.0,
    "c": {},
    "c_k_max": {k: 2.0 for k in range(1,K+1)},
    "d_init": {k: 0.0 for k in range(1,K+1)},
    "b_init": {k: 1000.0 for k in range(1,K+1)},
    "C": 2.0,
    "target_coords": {t['p']:(t['lat'], t['lon']) for t in targets},
    "ground_stations": ground_stations
}

outfn = "reossp_sandy_data.pkl"
with open(outfn, "wb") as f:
    pickle.dump(data, f)
print("Saved data to", outfn)


## Step 7 — Run the toy solver using the saved data

1. Copy the Gurobi toy solver script `reossp_rhp_gurobi_toy.py` (provided earlier) into the same folder.
2. In the solver code, replace the call to `generate_toy_instance()` with loading the pickle file:

```python
import pickle
with open('reossp_sandy_data.pkl','rb') as f:
    data = pickle.load(f)
```

3. Adjust `L` (lookahead length) and other parameters as needed, then run the solver script.

If you prefer the Pyomo script, you can similarly load the pickle into that script and call `rolling_horizon_solve(data, L=...)`.

---

### Final notes

- This notebook provides a reproducible, runnable pipeline to _prepare_ inputs described in the paper. For research-grade
  experiments you should substitute the simple orbit generator with SGP4 propagation (Python `sgp4` package or `poliastro`) and
  compute transfer costs `c[(s,k,i,j)]` using orbital mechanics formulas or trajectory optimization.
- If you want, I can now:
  - (A) Add SGP4-based propagation cells to compute V/W/H precisely (requires TLEs for satellites and internet or TLE files), or
  - (B) Help you adapt the Gurobi/Pyomo solver script to load `reossp_sandy_data.pkl` and run the experiment.

Tell me which next step you want (A or B).
